In [1]:
import sys
import os
import json
from collections import Counter
from konlpy.tag import Okt, Komoran, Hannanum, Kkma, Mecab
import tkinter as tk
from tkinter import filedialog, messagebox, ttk, StringVar
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib import font_manager, rc
from matplotlib.ticker import FuncFormatter
from openpyxl import Workbook
import ijson
import csv


# 단어 통계를 위한 Counter 객체 생성
word_counter = Counter()

mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")

# 형태소 분석기 초기화
morpheme_analyzers = {
    "선택 없음": None,
    "Okt": Okt(),
    "Komoran": Komoran(),
    "Hannanum": Hannanum(),
    "Kkma": Kkma(),
    'Mecab': mecab
}


# 폰트 설정
#if getattr(sys, 'frozen', False):  # 코드가 PyInstaller로 패키징된 경우
    #base_path = sys._MEIPASS
#else:
#    base_path = os.path.dirname(__file__)

base_path = os.getcwd()

font_path = os.path.join(base_path, "fonts", "malgun.ttf")
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)


def generate_ngrams(s, n):
    # Input: s = string, n = size of the ngram
    # Output: list of ngrams
    tokens = s.split()
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]


all_sentences = {'original': [], 'analyzed': []}


def analyze_data(data, path_elements, target_key, target_value):
    ngram_size = int(ngram_cnt_entry.get())

    if not path_elements:  # 더 이상 분석할 태그가 없는 경우
        if isinstance(data, str):  # 데이터가 문자열인 경우
            # concordance_entry에서 단어 목록을 가져옵니다.
            concordance_words = concordance_entry.get().split('|')
            concordance_words = [word.strip() for word in concordance_words]
            # 원본 문장 저장
            if any(con_word in data for con_word in concordance_words):
                all_sentences['original'].append(data)

            # 제외 단어 목록을 가져옵니다.
            exclude_words = exclude_words_entry.get().split('|')
            exclude_words = [word.strip() for word in exclude_words]

            # 형태소 분석기가 선택되지 않은 경우 n-gram 분석만 진행
            selected_analyzer = morpheme_analyzer.get()
            ngrams = []
            if any(con_word in data for con_word in concordance_words):
                if not selected_analyzer or selected_analyzer == "선택 없음" or selected_analyzer not in morpheme_analyzers:
                        words = data.split()  # 문장을 단어로 분리합니다.
                        filtered_words = [word for word in words if word not in exclude_words]  # 제외 단어를 걸러냅니다.
                        ngrams = generate_ngrams(' '.join(filtered_words), ngram_size)  # n-gram을 생성합니다.
                        all_sentences['analyzed'].append("")  # 혹은 None
                else:
                    # 선택한 형태소 분석기로 문장을 형태소 분석합니다.
                    morphemes = morpheme_analyzers[morpheme_analyzer.get()].pos(data)
                    # 제외 단어 목록에 포함되지 않은 형태소만 추가합니다.
                    filtered_morphemes = [f"{word}/{tag}" for word, tag in morphemes if word not in exclude_words]
                    # 문장을 형태소 분석된 형태로 변환합니다.
                    analyzed_sentence = ' '.join(filtered_morphemes)
                    # 분석된 문장 저장
                    all_sentences['analyzed'].append(analyzed_sentence)
                    # n-gram을 생성합니다.
                    ngrams = generate_ngrams(analyzed_sentence, ngram_size)

            # n-gram을 Counter에 추가합니다.
            for ngram in ngrams:
                word_counter[ngram] = word_counter.get(ngram, {'count': 0})
                word_counter[ngram]['count'] += 1
        return

    current_element = path_elements[0]
    remaining_elements = path_elements[1:]

    # data가 리스트인 경우 각 요소에 대해 analyze_data를 호출합니다.
    if isinstance(data, list):
        keys = target_key.split('.')
        for item in data:
            if isinstance(item, dict):
                value = None
                targetdata = item
                if keys[0] != '':
                    for i, key in enumerate(keys):
                        if i == len(keys) - 1:
                            value = targetdata.get(key)
                        else:
                            targetdata = targetdata.get(key, {})

                if value is not None:
                    if target_value not in value:
                        # 사용자가 입력한 target_value와 다른 경우, 해당 항목의 분석을 중단하고 다음 항목으로 넘어갑니다.
                        continue

                sub_data = item.get(current_element, {})
                analyze_data(sub_data, remaining_elements, target_key, target_value)

    # data가 딕셔너리인 경우 current_element 태그를 가져와 analyze_data를 호출합니다.
    elif isinstance(data, dict):
        sub_data = data.get(current_element, {})
        analyze_data(sub_data, remaining_elements, target_key, target_value)


# 그래프를 그린 canvas 객체
canvas = None


def plot_data():
    global canvas

    try:
        # Get the number of items to display from the entry widget
        number_of_items = int(graph_cnt_entry.get())
    except ValueError:  # In case of invalid input
        messagebox.showerror("Error", "Invalid number entered for graph count.")
        return

    # Create new figure
    fig = Figure(figsize=(8, 6), dpi=100)

    # Add a subplot to the new figure
    ax = fig.add_subplot(1, 1, 1)

    # Get the most common 'number_of_items' words
    common_words = sorted(word_counter.items(), key=sort_by_count, reverse=True)[:number_of_items]

    # Separate the words and their counts
    words = [word for word, data in common_words]
    counts = [data['count'] for word, data in common_words]

    # Plot the data
    ax.bar(words, counts)

    # Adjust the x-axis labels
    ax.set_xticks(words)
    shortened_labels = [label if len(label) <= 10 else label[:10] + "..." for label in words]
    ax.set_xticklabels(shortened_labels, rotation=45, ha="right", fontsize=8)

    def hide_non_integers(x, pos):
        if x.is_integer():
            return "{:.0f}".format(x)
        return ""

    def integer_ticks(ax):
        # 현재 Y축의 눈금 위치를 가져옵니다.
        ticks = ax.get_yticks()

        # 소수점이 포함된 눈금 위치를 제거합니다.
        int_ticks = [tick for tick in ticks if tick.is_integer()]

        # Y축의 눈금 위치를 정수만 포함하도록 설정합니다.
        ax.set_yticks(int_ticks)

    # Set y-axis tick labels to show only integers
    ax.get_yaxis().set_major_formatter(FuncFormatter(hide_non_integers))
    integer_ticks(ax)

    fig.tight_layout()

    # Remove previous canvas if exists
    if canvas is not None:
        canvas.get_tk_widget().pack_forget()
        canvas = None

    # Create a new tkinter Canvas containing the figure
    canvas = FigureCanvasTkAgg(fig, master=graph_frame)
    canvas.draw()

    # Add the canvas to the Label widget
    canvas.get_tk_widget().pack()
    # canvas에 우클릭 이벤트 바인딩
    canvas.get_tk_widget().bind("<Button-3>", save_figure_as_image)


# 그래프를 이미지로 저장하는 함수
def save_figure_as_image(event):
    # 파일 저장 대화상자를 열어 이미지 저장 위치를 지정
    file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                              filetypes=[("PNG files", "*.png"),
                                                         ("JPEG files", "*.jpg"),
                                                         ("All files", "*.*")])
    if file_path:
        # matplotlib의 savefig 메소드를 사용하여 이미지로 저장
        canvas.figure.savefig(file_path)
        # 저장된 이미지 파일 열기
        if messagebox.askyesno("Information", "저장되었습니다. 파일을 열겠습니까?"):
            os.startfile(file_path)


def sort_by_count(item):
    return item[1]['count']


def user_input_to_ijson_prefix(user_input_path):
    """Convert user input path to ijson prefix format."""
    parts = user_input_path.split('.')
    ijson_parts = []
    for part in parts:
        ijson_parts.append(part)
        ijson_parts.append('item')
    # Remove the last 'item'
    ijson_parts.pop()
    return '.'.join(ijson_parts)


def analyze_folder():
    global word_counter, all_sentences

    folder_path = filedialog.askdirectory()
    if not folder_path:
        return

    user_input_path = user_input.get()
    path_elements = user_input_path.split('.')

    # GUI에서 필터링할 key와 value를 가져옵니다.
    user_target_key = filter_key_entry.get()
    user_target_value = filter_value_entry.get()

    # 폴더 내 json 파일 개수를 세어 프로그레스바 최대값 설정
    total_files = len([name for name in os.listdir(folder_path) if name.endswith('.json')])
    progress_bar['maximum'] = total_files

    # 초기화
    word_counter = Counter()
    all_sentences = {'original': [], 'analyzed': []}
    reset_table()

    processed_files = 0
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)

            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                analyze_data(data, path_elements, user_target_key, user_target_value)

            # 파일 하나 처리가 끝나면 프로그레스바를 업데이트
            processed_files += 1
            progress_bar['value'] = processed_files
            root.update_idletasks()

    # 사용자 입력에서 추출 수 가져오기
    extract_count = extract_cnt_entry.get()
    if extract_count:
        n = int(extract_count)
        sorted_items = sorted(word_counter.items(), key=lambda item: item[1]['count'], reverse=True)[:n]
    else:
        sorted_items = sorted(word_counter.items(), key=lambda item: item[1]['count'], reverse=True)

    # 결과값이 있는지 확인
    if not sorted_items:
        messagebox.showinfo("Information", "조건에 해당하는 결과값이 없습니다!")
        return

    # 결과 CSV 파일 경로 설정
    word_csv_path = os.path.join(folder_path, "word_count.csv")
    sentence_csv_path = os.path.join(folder_path, "sentence_analysis.csv")

    # 단어 카운터를 CSV 파일로 저장
    with open(word_csv_path, 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Word", "Count"])
        for word, count_data in sorted_items:
            writer.writerow([word, count_data['count']])

    # 문장 분석 결과를 CSV 파일로 저장
    with open(sentence_csv_path, 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Original", "Analyzed"])
        for original, analyzed in zip(all_sentences['original'], all_sentences['analyzed']):
            writer.writerow([original, analyzed])

    plot_data()  # Plot the data

    messagebox.showinfo("Information", f"형태소 분석이 완료되었습니다! 결과가 '{folder_path}'에 저장되었습니다.")


def reset_table():
    global canvas, word_counter

    # Remove previous canvas if exists
    if canvas is not None:
        canvas.get_tk_widget().pack_forget()
        canvas = None

    # 표를 비웁니다.
    # result_tree.delete(*result_tree.get_children())

    # 단어 카운터를 초기화합니다.
    word_counter = Counter()

    # 프로그레스바를 0으로 초기화합니다.
    progress_bar['value'] = 0


def update_user_input(*args):
    user_input.delete(0, 'end')
    user_input.insert(0, tag_options[tag_variable.get()])


def get_selected_data():
    output_type = output_option_combobox.get()
    data = []

    if output_type == "단어":
        for item in result_tree.get_children():
            word, count = result_tree.item(item, 'values')
            data.append((word, count))
    elif output_type == "문장":
        for original, analyzed in zip(all_sentences['original'], all_sentences['analyzed']):
            data.append((original, analyzed))

    return data


def save_to_excel():
    selected_data = get_selected_data()

    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv"), ("All Files", "*.*")])
    if file_path:
        with open(file_path, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.writer(csvfile)

            # 헤더 작성
            if output_option_combobox.get() == "단어":
                writer.writerow(["Word", "Count"])
            else:  # 문장
                writer.writerow(["Original", "Analyzed"])

            # 데이터 작성
            for data in selected_data:
                writer.writerow(data)

        if messagebox.askyesno("Information", "저장되었습니다. 파일을 열겠습니까?"):
            os.startfile(file_path)


def save_to_txt():
    selected_data = get_selected_data()

    file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt"), ("All Files", "*.*")])

    if file_path:
        with open(file_path, 'w', encoding="utf-8") as f:
            if output_option_combobox.get() == "단어":
                f.write("Word\tCount\n")
                for word, count in selected_data:
                    f.write(f"{word}\t{count}\n")
            else:  # 문장
                f.write("Original\tAnalyzed\n")
                for original, analyzed in selected_data:
                    f.write(f"{original}\t{analyzed}\n")

        if messagebox.askyesno("Information", "저장되었습니다. 파일을 열겠습니까?"):
            os.startfile(file_path)


# 스타일 설정
def configure_styles():
    style = ttk.Style()
    style.theme_use('clam')  # 클램 테마는 더 현대적인 느낌을 줍니다.
    style.configure('TLabel', font=('Arial', 10), background='white')
    style.configure('TEntry', font=('Arial', 10), padding=5)
    style.configure('TButton', font=('Arial', 10), padding=5)
    style.configure('TCombobox', font=('Arial', 10), padding=5)
    style.map('TCombobox', fieldbackground=[('readonly', 'white')],
              selectbackground=[('readonly', 'white')],
              selectforeground=[('readonly', 'black')])
    style.configure('TFrame', background='white')  # 프레임 배경색 설정
    style.configure('Horizontal.TProgressbar', background='#FA8072')


def only_numbers(char):
    return char.isdigit()


root = tk.Tk()
root.title("n-gram 및 형태소 분석기 v1.1")
# 프로그램의 고정된 크기
program_width = 670
program_height = 850

# 화면의 중앙에 프로그램이 위치하도록 좌표를 계산합니다.
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
center_x = int((screen_width - program_width) / 2)
center_y = int((screen_height - program_height) / 2)

# 프로그램의 위치와 크기를 설정합니다.
root.geometry(f'{program_width}x{program_height}+{center_x}+{center_y}')

configure_styles()

# 숫자 입력 확인을 위한 유효성 검사 커맨드 생성
vcmd = root.register(only_numbers)

# (1,1)
tag_label = tk.Label(root, text="* 태그", anchor='w')
tag_label.grid(row=0, column=0, sticky='we', padx=10, pady=5)

# (1,2)
user_input = ttk.Entry(root)
user_input.grid(row=0, column=1, sticky='ew', padx=10, pady=5)

# (1,3)
tag_menu_label = tk.Label(root, text="* 태그 선택", anchor='w')
tag_menu_label.grid(row=0, column=2, sticky='we', padx=10, pady=5)

# (1,4)
tag_variable = StringVar(root)
tag_variable.trace("w", update_user_input)
tag_options = {
    "신문 말뭉치": "document.paragraph.form",
    "일상 대화 말뭉치": "document.utterance.form",
    "직접 입력": "",
}
tag_menu = ttk.Combobox(root, textvariable=tag_variable, values=list(tag_options.keys()), state='readonly')
tag_menu.grid(row=0, column=3, sticky='ew', padx=10, pady=5)
tag_menu.set("직접 입력")

# (2,1)
ngram_cnt_label = tk.Label(root, text="* n-gram 사이즈", anchor='w')
ngram_cnt_label.grid(row=1, column=0, sticky='we', padx=10, pady=5)

# (2,2)
ngram_cnt_entry = ttk.Entry(root, validate="key", validatecommand=(vcmd, '%S'))
ngram_cnt_entry.grid(row=1, column=1, sticky='ew', padx=10, pady=5)
ngram_cnt_entry.insert(0, "1")

# (2,3)
tag_menu_label = tk.Label(root, text="* 형태소 분석기", anchor='w')
tag_menu_label.grid(row=1, column=2, sticky='we', padx=10, pady=5)

# (2,4)
morpheme_analyzer = StringVar(root)
morpheme_menu = ttk.Combobox(root, textvariable=morpheme_analyzer, values=list(morpheme_analyzers.keys()), state='readonly')
morpheme_menu.grid(row=1, column=3, sticky='ew', padx=10, pady=5)
morpheme_menu.set("선택 없음")

# (3,1)
graph_cnt_label = tk.Label(root, text="* 그래프 표출수(최대20)", anchor='w')
graph_cnt_label.grid(row=2, column=0, sticky='we', padx=10, pady=5)

# (3,2)
graph_cnt_entry = ttk.Entry(root, validate="key", validatecommand=(vcmd, '%S'))
graph_cnt_entry.grid(row=2, column=1, sticky='ew', padx=10, pady=5)
graph_cnt_entry.insert(0, "10")

# (3,3)
extract_cnt_label = tk.Label(root, text="표 표출수", anchor='w')
extract_cnt_label.grid(row=2, column=2, sticky='we', padx=10, pady=5)

# (3,4)
extract_cnt_entry = ttk.Entry(root, validate="key", validatecommand=(vcmd, '%S'))
extract_cnt_entry.grid(row=2, column=3, sticky='ew', padx=10, pady=5)

# (4,1)
filter_key_label = tk.Label(root, text="필터키", anchor='w')
filter_key_label.grid(row=3, column=0, sticky='we', padx=10, pady=5)

# (4,2)
filter_key_entry = ttk.Entry(root)
filter_key_entry.grid(row=3, column=1, sticky='ew', padx=10, pady=5)

# (4,3)
tag_menu_label = tk.Label(root, text="필터값", anchor='w')
tag_menu_label.grid(row=3, column=2, sticky='we', padx=10, pady=5)

# (4,4)
filter_value_entry = ttk.Entry(root)
filter_value_entry.grid(row=3, column=3, sticky='ew', padx=10, pady=5)

# (5,1)
concordance_label = tk.Label(root, text="콘코던스 단어(|로 구분)", anchor='w')
concordance_label.grid(row=4, column=0, sticky='we', padx=10, pady=5)

# (5,2)
concordance_entry = ttk.Entry(root)
concordance_entry.grid(row=4, column=1, sticky='ew', padx=10, pady=5)

# (5,3)
exclude_words_label = tk.Label(root, text="제외 단어(|로 구분)", anchor='w')
exclude_words_label.grid(row=4, column=2, sticky='we', padx=10, pady=5)

# (5,4)
exclude_words_entry = ttk.Entry(root)
exclude_words_entry.grid(row=4, column=3, sticky='ew', padx=10, pady=5)

# (6,1)
copyright_label = tk.Label(root, text="COPYRIGHTⓒ2023 (주)싸인랩 ALL RIGHTS RESERVED.", anchor='w')
copyright_label.grid(row=5, column=0, columnspan=2, sticky='we', padx=10, pady=5)

# (6,3)
folder_button = ttk.Button(root, text="폴더 선택 및 분석", command=analyze_folder)
folder_button.grid(row=5, column=2, sticky='ew', padx=10, pady=5)

# (6,4)
reset_button = ttk.Button(root, text="결과창 리셋", command=reset_table)
reset_button.grid(row=5, column=3, sticky='ew', padx=10, pady=5)

# (7,1)
progress_bar = ttk.Progressbar(root, orient="horizontal", length=200, mode="determinate")
progress_bar.grid(row=6, column=0, columnspan=4, sticky='we')

# (사용안함)
main_frame = ttk.Frame(root, height=400)  # height를 설정해 줍니다.
main_frame.grid(row=7, column=0, columnspan=4, sticky='ew')  # sticky를 'ew'로 변경합니다.
buttons_frame = ttk.Frame(main_frame)

excel_download_button = tk.Button(buttons_frame, text="엑셀 다운", command=save_to_excel)
excel_download_button.pack(side=tk.RIGHT)
text_download_button = tk.Button(buttons_frame, text="텍스트 다운", command=save_to_txt)
text_download_button.pack(side=tk.RIGHT)
# 출력옵션 추가
output_option_combobox = ttk.Combobox(buttons_frame, values=["단어", "문장"], width=5, state='readonly')
output_option_combobox.pack(side=tk.RIGHT, fill=tk.Y)
output_option_combobox.set("단어")  # 기본값 설정

# (8,1)
graph_frame = ttk.Frame(root, height=600)  # height를 설정해 줍니다.
graph_frame.grid(row=7, column=0, columnspan=4, sticky='ew')  # sticky를 'ew'로 변경합니다.

# 스크롤바 추가
scrollbar = ttk.Scrollbar(main_frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# 결과 출력을 위한 표
buttons_frame.pack(side=tk.TOP, fill=tk.X, padx=5)
result_tree = ttk.Treeview(main_frame, columns=("Word", "Count"), show='headings', yscrollcommand=scrollbar.set, height=7)
result_tree.heading("#1", text="Word")
result_tree.heading("#2", text="Count")
result_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, pady=(5, 0))

# 스크롤바와 표 연결
scrollbar.config(command=result_tree.yview)
root.grid_columnconfigure(1, weight=1)
root.grid_rowconfigure(6, weight=1)
root.grid_rowconfigure(7, weight=1)

root.mainloop()

In [2]:
# 스타일 설정
def configure_styles():
    style = ttk.Style()
    style.theme_use('clam')  # 클램 테마는 더 현대적인 느낌을 줍니다.
    style.configure('TLabel', font=('Arial', 10), background='white')
    style.configure('TEntry', font=('Arial', 10), padding=5)
    style.configure('TButton', font=('Arial', 10), padding=5)
    style.configure('TCombobox', font=('Arial', 10), padding=5)
    style.map('TCombobox', fieldbackground=[('readonly', 'white')],
              selectbackground=[('readonly', 'white')],
              selectforeground=[('readonly', 'black')])
    style.configure('TFrame', background='white')  # 프레임 배경색 설정
    style.configure('Horizontal.TProgressbar', background='#FA8072')

In [3]:
root = tk.Tk()
root.title("n-gram 및 형태소 분석기 v1.1")
# 프로그램의 고정된 크기
program_width = 670
program_height = 850

# 화면의 중앙에 프로그램이 위치하도록 좌표를 계산합니다.
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
center_x = int((screen_width - program_width) / 2)
center_y = int((screen_height - program_height) / 2)

# 프로그램의 위치와 크기를 설정합니다.
root.geometry(f'{program_width}x{program_height}+{center_x}+{center_y}')

configure_styles()

# 숫자 입력 확인을 위한 유효성 검사 커맨드 생성
vcmd = root.register(only_numbers)

# (1,1)
tag_label = tk.Label(root, text="* 태그", anchor='w')
tag_label.grid(row=0, column=0, sticky='we', padx=10, pady=5)

# (1,2)
user_input = ttk.Entry(root)
user_input.grid(row=0, column=1, sticky='ew', padx=10, pady=5)

# (1,3)
tag_menu_label = tk.Label(root, text="* 태그 선택", anchor='w')
tag_menu_label.grid(row=0, column=2, sticky='we', padx=10, pady=5)

# (1,4)
tag_variable = StringVar(root)
tag_variable.trace("w", update_user_input)
tag_options = {
    "신문 말뭉치": "document.paragraph.form",
    "일상 대화 말뭉치": "document.utterance.form",
    "직접 입력": "",
}
tag_menu = ttk.Combobox(root, textvariable=tag_variable, values=list(tag_options.keys()), state='readonly')
tag_menu.grid(row=0, column=3, sticky='ew', padx=10, pady=5)
tag_menu.set("직접 입력")

# (2,1)
ngram_cnt_label = tk.Label(root, text="* n-gram 사이즈", anchor='w')
ngram_cnt_label.grid(row=1, column=0, sticky='we', padx=10, pady=5)

# (2,2)
ngram_cnt_entry = ttk.Entry(root, validate="key", validatecommand=(vcmd, '%S'))
ngram_cnt_entry.grid(row=1, column=1, sticky='ew', padx=10, pady=5)
ngram_cnt_entry.insert(0, "1")

# (2,3)
tag_menu_label = tk.Label(root, text="* 형태소 분석기", anchor='w')
tag_menu_label.grid(row=1, column=2, sticky='we', padx=10, pady=5)

# (2,4)
morpheme_analyzer = StringVar(root)
morpheme_menu = ttk.Combobox(root, textvariable=morpheme_analyzer, values=list(morpheme_analyzers.keys()), state='readonly')
morpheme_menu.grid(row=1, column=3, sticky='ew', padx=10, pady=5)
morpheme_menu.set("선택 없음")

# (3,1)
graph_cnt_label = tk.Label(root, text="* 그래프 표출수(최대20)", anchor='w')
graph_cnt_label.grid(row=2, column=0, sticky='we', padx=10, pady=5)

# (3,2)
graph_cnt_entry = ttk.Entry(root, validate="key", validatecommand=(vcmd, '%S'))
graph_cnt_entry.grid(row=2, column=1, sticky='ew', padx=10, pady=5)
graph_cnt_entry.insert(0, "10")

# (3,3)
extract_cnt_label = tk.Label(root, text="표 표출수", anchor='w')
extract_cnt_label.grid(row=2, column=2, sticky='we', padx=10, pady=5)

# (3,4)
extract_cnt_entry = ttk.Entry(root, validate="key", validatecommand=(vcmd, '%S'))
extract_cnt_entry.grid(row=2, column=3, sticky='ew', padx=10, pady=5)

# (4,1)
filter_key_label = tk.Label(root, text="필터키", anchor='w')
filter_key_label.grid(row=3, column=0, sticky='we', padx=10, pady=5)

# (4,2)
filter_key_entry = ttk.Entry(root)
filter_key_entry.grid(row=3, column=1, sticky='ew', padx=10, pady=5)

# (4,3)
tag_menu_label = tk.Label(root, text="필터값", anchor='w')
tag_menu_label.grid(row=3, column=2, sticky='we', padx=10, pady=5)

# (4,4)
filter_value_entry = ttk.Entry(root)
filter_value_entry.grid(row=3, column=3, sticky='ew', padx=10, pady=5)

# (5,1)
concordance_label = tk.Label(root, text="콘코던스 단어(|로 구분)", anchor='w')
concordance_label.grid(row=4, column=0, sticky='we', padx=10, pady=5)

# (5,2)
concordance_entry = ttk.Entry(root)
concordance_entry.grid(row=4, column=1, sticky='ew', padx=10, pady=5)

# (5,3)
exclude_words_label = tk.Label(root, text="제외 단어(|로 구분)", anchor='w')
exclude_words_label.grid(row=4, column=2, sticky='we', padx=10, pady=5)

# (5,4)
exclude_words_entry = ttk.Entry(root)
exclude_words_entry.grid(row=4, column=3, sticky='ew', padx=10, pady=5)

# (6,1)
copyright_label = tk.Label(root, text="COPYRIGHTⓒ2023 (주)싸인랩 ALL RIGHTS RESERVED.", anchor='w')
copyright_label.grid(row=5, column=0, columnspan=2, sticky='we', padx=10, pady=5)

# (6,3)
folder_button = ttk.Button(root, text="폴더 선택 및 분석", command=analyze_folder)
folder_button.grid(row=5, column=2, sticky='ew', padx=10, pady=5)

# (6,4)
reset_button = ttk.Button(root, text="결과창 리셋", command=reset_table)
reset_button.grid(row=5, column=3, sticky='ew', padx=10, pady=5)

# (7,1)
progress_bar = ttk.Progressbar(root, orient="horizontal", length=200, mode="determinate")
progress_bar.grid(row=6, column=0, columnspan=4, sticky='we')

# (사용안함)
main_frame = ttk.Frame(root, height=400)  # height를 설정해 줍니다.
main_frame.grid(row=7, column=0, columnspan=4, sticky='ew')  # sticky를 'ew'로 변경합니다.
buttons_frame = ttk.Frame(main_frame)

excel_download_button = tk.Button(buttons_frame, text="엑셀 다운", command=save_to_excel)
excel_download_button.pack(side=tk.RIGHT)
text_download_button = tk.Button(buttons_frame, text="텍스트 다운", command=save_to_txt)
text_download_button.pack(side=tk.RIGHT)
# 출력옵션 추가
output_option_combobox = ttk.Combobox(buttons_frame, values=["단어", "문장"], width=5, state='readonly')
output_option_combobox.pack(side=tk.RIGHT, fill=tk.Y)
output_option_combobox.set("단어")  # 기본값 설정

# (8,1)
graph_frame = ttk.Frame(root, height=600)  # height를 설정해 줍니다.
graph_frame.grid(row=7, column=0, columnspan=4, sticky='ew')  # sticky를 'ew'로 변경합니다.

# 스크롤바 추가
scrollbar = ttk.Scrollbar(main_frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# 결과 출력을 위한 표
buttons_frame.pack(side=tk.TOP, fill=tk.X, padx=5)
result_tree = ttk.Treeview(main_frame, columns=("Word", "Count"), show='headings', yscrollcommand=scrollbar.set, height=7)
result_tree.heading("#1", text="Word")
result_tree.heading("#2", text="Count")
result_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, pady=(5, 0))

# 스크롤바와 표 연결
scrollbar.config(command=result_tree.yview)
root.grid_columnconfigure(1, weight=1)
root.grid_rowconfigure(6, weight=1)
root.grid_rowconfigure(7, weight=1)